In [1]:
#@title 🛠️ Install all required packages
#@markdown Do not run `Restart runtime` pop up if this code cell is not completed (around 3 minutes on GPU T4)

# === Core ML & NLP Libraries ===
!pip install transformers==4.41.2
!pip install sentence-transformers==3.0.1
!pip install scikit-learn==1.5.0
!pip install accelerate==0.31.0
!pip install peft==0.11.1

# === Scientific Computing (compatible with Python 3.12) ===
!pip install numpy==1.26.4
!pip install scipy==1.11.4  # Changed: 1.10.1 not available for Python 3.12

# === Topic Modeling & NLP ===
!pip install gensim==4.3.3
!pip install datasets
!pip install bertopic==0.16.0
!pip install datamapplot

# === Vector Search & Retrieval ===
!pip install faiss-cpu==1.8.0
!pip install cohere==5.5.8
!pip install rank_bm25==0.2.2

# === LangChain Ecosystem (downgraded for legacy memory support) ===
!pip install langchain==0.1.20
!pip install langchain-community==0.0.38
!pip install langchain-core==0.1.52
!pip install langchain-openai==0.1.7

# === Llama Integration ===
!pip install llama-cpp-python==0.2.78 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

  Using cached scikit_learn-1.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.2
    Uninstalling scikit-learn-1.7.2:
      Successfully uninstalled scikit-learn-1.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.0 which is incompatible.
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
  Attemp

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


## Claude debugging steps

Step 1: Attempted scipy downgrade

Action: Uninstalled and reinstalled scipy==1.10.1 with --no-cache-dir
Result: ❌ Same error persisted
Error: ImportError: cannot import name 'triu' from 'scipy.linalg'

Step 2: Upgraded Gensim

Action: Upgraded gensim from 4.3.2 to 4.3.3
Command: !pip install gensim==4.3.3
Result: ✅ Scipy/Gensim compatibility issue resolved
Root Cause: Gensim 4.3.2 was incompatible with newer SciPy versions that removed triu

Step 3: Installed LangChain packages

Action: Installed langchain==0.2.5 and langchain-community==0.2.5
Command: !pip install langchain==0.2.5 langchain-community==0.2.5
Result: ✅ ModuleNotFoundError: No module named 'langchain_community' resolved

Step 4: Installed langchain-openai

Action: Installed langchain-openai package
Command: !pip install langchain-openai
Result: ✅ ModuleNotFoundError: No module named 'langchain_openai' resolved

Step 5: Upgraded langchain-core

Action: Upgraded langchain-core to resolve pydantic compatibility
Command: !pip install langchain-core --upgrade
Result: ✅ ModuleNotFoundError: No module named 'langchain_core.pydantic_v1' resolved

Step 6: Upgraded all LangChain packages

Action: Upgraded all langchain packages together for compatibility
Command: !pip install langchain langchain-core langchain-community langchain-openai --upgrade
Result: ✅ ImportError: cannot import name 'is_data_content_block' resolved

Step 7: Changed memory import location

Action: Changed import from langchain.memory to langchain_community.memory
Result: ❌ Still failed - memory classes not found in that location
Error: ImportError: cannot import name 'ConversationBufferMemory' from 'langchain_community.memory'

Step 8: Downgraded to LangChain 0.1.x (FINAL SOLUTION)

Action: Downgraded to older LangChain versions that support legacy memory classes
Command: !pip install langchain==0.1.20 langchain-community==0.0.38 langchain-core==0.1.52 langchain-openai==0.1.7
Import: Kept original import from langchain.memory import ...
Result: ✅ All imports working!

In [1]:
#@title 🚚 Import all libraries
# === Core Python Utilities ===
import os
import json
import random
import subprocess
from copy import deepcopy
from urllib import request

# === Progress & Visualization ===
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud

# === Data Handling & Analysis ===
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import (
    RidgeClassifier,
    SGDClassifier,
    Perceptron,
    PassiveAggressiveClassifier,
    LogisticRegression
)
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier
)

# === Transformers & NLP Models ===
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    pipeline,
    logging
)
from sentence_transformers import SentenceTransformer
import gensim.downloader as api
from gensim.models import Word2Vec

# === Topic Modeling & Embedding Tools ===
from bertopic import BERTopic
from bertopic.representation import (KeyBERTInspired, MaximalMarginalRelevance, TextGeneration, OpenAI)
from umap import UMAP
from hdbscan import HDBSCAN
import datamapplot

# === Datasets & API Clients ===
from datasets import load_dataset
import openai
from google.colab import userdata

# === LangChain & Llama Integrations ===
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import (ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory)
from langchain import LlamaCpp, PromptTemplate, LLMChain # Llama
from langchain.agents import (load_tools, Tool, AgentExecutor, create_react_agent)
from langchain.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI

# === Transformers Utils ===
from transformers.pipelines.pt_utils import KeyDataset

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [2]:
#@title 🕸️ Global Config
#@markdown This is the global config applied downstream, feel free to adjust the models based on the proposed list
MODEL = 'microsoft/Phi-3-mini-4k-instruct' #@param ['google/gemma-2-2b-it', 'microsoft/Phi-3-mini-4k-instruct', 'Qwen/Qwen2.5-3B-Instruct', 'microsoft/deberta-v3-xsmall', 'google/flan-t5-small' ]
SENTENCE_TRANSFORMER_MODEL = "all-mpnet-base-v2" #@param ['thenlper/gte-small', 'all-mpnet-base-v2']
TEXT_CLASSIFICATION_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest" #@param {type:'string'}
DEVICE = 'cuda' #@param ['cuda', 'cpu']
SHOW_LIBRARIES_VERSIONS = True #@param {type:'boolean'}

assert userdata.get("openai_apikey"), "Please specify the Colab secret `openai_apikey` to run code cells pointing to the openai models "
assert userdata.get("cohere_apikey"), "Please specify the Colab secret `cohere_apikey` to use Cohere API"

In [3]:
#@title 📗 Show versions of libraries
#@markdown Investigate which versions are there. There are many packages installed in the runtime
#@markdown so in case of issues with dependencies - this is the place to start debugging

packages = [
    "transformers", "sentence-transformers", "gensim", "scikit-learn",
    "accelerate", "peft", "scipy", "numpy", "datasets", "bertopic",
    "datamapplot", "llama-cpp-python", "langchain", "faiss-cpu",
    "cohere", "langchain-community", "rank_bm25"
]

print("\n🔍 Checking installed library versions...\n")

if SHOW_LIBRARIES_VERSIONS:
    for pkg in packages:
        try:
            result = subprocess.run(['pip', 'show', pkg], capture_output=True, text=True)
            if result.returncode == 0 and result.stdout.strip():
                info = {}
                for line in result.stdout.splitlines():
                    if ": " in line:
                        key, value = line.split(": ", 1)
                        info[key.strip()] = value.strip()
                print(f"📦 {info.get('Name', pkg)} == {info.get('Version', 'Unknown')}")
                print(f"    Summary : {info.get('Summary', 'No description')}\n")
            else:
                print(f"⚠️ {pkg} not found or not installed.\n")
        except Exception as e:
            print(f"❌ Error checking {pkg}: {e}\n")


🔍 Checking installed library versions...

📦 transformers == 4.41.2
    Summary : State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow

📦 sentence-transformers == 3.0.1
    Summary : Multilingual text embeddings

📦 gensim == 4.3.3
    Summary : Python framework for fast Vector Space Modelling

📦 scikit-learn == 1.7.2
    Summary : A set of python modules for machine learning and data mining

📦 accelerate == 0.31.0
    Summary : Accelerate

📦 peft == 0.11.1
    Summary : Parameter-Efficient Fine-Tuning (PEFT)

📦 libquadmath == 1.11.4
    Summary : Fundamental algorithms for scientific computing in Python

📦 libquadmath == 1.26.4
    Summary : Fundamental package for array computing in Python

📦 datasets == 4.0.0
    Summary : HuggingFace community-driven open-source library of datasets

📦 bertopic == 0.16.0
    Summary : BERTopic performs topic Modeling with state-of-the-art transformer models.

📦 datamapplot == 0.6.4
    Summary : A library for presentation and publication

In [4]:
#@title 🤖 Setting up all models used downstream
#@markdown Models are subjectively chosen, feel free to adjust the list in the MODELS param below
#@markdown <b>AutoModelForCausalLM</b>: Loads a model specifically designed for causal language modeling (text generation). It includes a language modeling head that outputs logits over the vocabulary.
#@markdown <br><b>AutoModel</b>: Loads the base model architecture without any task-specific head. It only returns the raw hidden states/embeddings from the transformer layers.
#
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

sentence_transformer_model = SentenceTransformer(f'sentence-transformers/{SENTENCE_TRANSFORMER_MODEL}')


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#@title 🛠️ User defined helper functions
#@markdown * describe_tokenizer
#@markdown * sentence_to_tokens_plot
#@markdown * evaluate_performance

def describe_tokenizer(tokenizer, n_tokens = 20):

    "Function that takes in a tokenizer object and prints some basic data about it"

    random_tokens = random.sample(range(tokenizer.vocab_size), 20)

    print(f"Description of tokenizer: {tokenizer.name_or_path}\n")
    print(f"Vocab size: {tokenizer.vocab_size}\n")
    print(f"Example {n_tokens} tokens:\n")

    decoded = [tokenizer.decode([n]) for n in random_tokens]
    print(" || ".join(decoded))

def sentence_to_tokens_plot(sentence, tokenizer):
    "Plot how the LLM connverted a human-language sentence to tokens"
    tokenizer = AutoTokenizer.from_pretrained(tokenizer)

    describe_tokenizer(tokenizer)

    input_token_ids = tokenizer(sentence).input_ids
    input_tokens = [tokenizer.decode(t) for t in input_token_ids]

    fig = px.imshow(np.array(input_token_ids).reshape(1, -1))
    fig.update_traces(text=np.array(input_tokens).reshape(1,-1),
        texttemplate="%{text}",
        textfont=dict(color="white", size=14)
    )
    fig.update_layout(title=f"Sentence: '{sentence}' | Tokenizer: {tokenizer.name_or_path}")
    fig.show()

def evaluate_performance(y_true, y_pred):
    """Remake the sklearn classification report to produce flattened dict
    that can be easily dumped into a pd.DataFrame"""

    perf_outputs = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"],
        output_dict=True
    )

    perf_outputs_flattened = {}

    for k in perf_outputs.keys():
        if isinstance(perf_outputs[k], dict):
            for sk in perf_outputs[k].keys():
                perf_outputs_flattened[f"{k}_{sk}".lower().replace(' ', '_')] = perf_outputs[k][sk]
        else:
            perf_outputs_flattened[k.lower().replace(' ', '_')] = perf_outputs[k]

    return perf_outputs_flattened


In [6]:
#@title 🌳 Setup of environment and logging variables

logging.set_verbosity_error() ## avoid device set to cuda warning
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

In [7]:
#@title 🔏 Describe tokenizer for your selected MODEL

describe_tokenizer(tokenizer)

Description of tokenizer: microsoft/Phi-3-mini-4k-instruct

Vocab size: 32000

Example 20 tokens:

копия || Tra || diagonal || SELECT || _. || owego || istik || Tu || Hotel || hnen || N || ASCII || inc || по || вест || Mod || donn || suggested || сооб || nú


In [8]:
#@title 🪄 Do predict without a pipeline by model.generate()

prompt = "Write me a single funny joke about cats and dogs. Don't explain it, just return the joke only <|assistant|>" #@param {type:'string'}

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate( input_ids=input_ids, max_new_tokens=100)

# Print the output

print(f"Input sentence: {prompt} \n")
print(f"Input tokens: {input_ids} \n")
print(f"Output sentence: {tokenizer.decode(generation_output[0])} \n")
print(f"Output tokens: {generation_output[0]} \n")

Input sentence: Write me a single funny joke about cats and dogs. Don't explain it, just return the joke only <|assistant|> 

Input tokens: tensor([[14350,   592,   263,  2323,  2090,  1460,  2958,   446,  1048,   274,
          1446,   322, 26361, 29889,  3872, 29915, 29873,  5649,   372, 29892,
           925,   736,   278,  2958,   446,   871, 29871, 32001]],
       device='cuda:0') 

Output sentence: Write me a single funny joke about cats and dogs. Don't explain it, just return the joke only <|assistant|> Why don't cats play poker in the jungle? Too many cheetahs!



<|endoftext|> 

Output tokens: tensor([14350,   592,   263,  2323,  2090,  1460,  2958,   446,  1048,   274,
         1446,   322, 26361, 29889,  3872, 29915, 29873,  5649,   372, 29892,
          925,   736,   278,  2958,   446,   871, 29871, 32001,  3750,  1016,
        29915, 29873,   274,  1446,  1708, 11293,   261,   297,   278,   432,
          686,   280, 29973,  1763, 29877,  1784,   923,   300,   801, 29879,


In [9]:
#@title 🪄 Wrap model in a pipeline and run a prompt for range of TEMPERATUREs.
#@markdown Here the code doesn't update per temperature

TEMPERATURES = [0.01, 0.1, 0.25, 0.33, 0.5, 0.75, 1] #@param
PROMPT = "Write short 4 lines of rhymed freestyle about why using LLMs is amazing." #@param {type:'string'}

for temperature in TEMPERATURES:

    generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    temperature=temperature,
    max_new_tokens=500,
    do_sample=True
    )

    messages = [{"role": "user", "content": PROMPT}]
    output = generator(messages)

    print('-' * 100)
    print(f"\nGenerated output by {MODEL} for temperature: {temperature}:\n")
    print('-' * 100)
    print(output[0]["generated_text"])



----------------------------------------------------------------------------------------------------

Generated output by microsoft/Phi-3-mini-4k-instruct for temperature: 0.01:

----------------------------------------------------------------------------------------------------
 In the realm of AI, LLMs shine so bright,

Unlocking knowledge with their vast insight.

From chatbots to translators, they're quite the sight,

Making our digital world feel just right.
----------------------------------------------------------------------------------------------------

Generated output by microsoft/Phi-3-mini-4k-instruct for temperature: 0.1:

----------------------------------------------------------------------------------------------------
 In the realm of data, LLMs are a sight to behold,
Crafting responses, stories, and knowledge untold.
With AI's embrace, our potential unfolds,
Amazing tools for the future, as we're bold.
----------------------------------------------------------------

In [10]:
#@title 🎨 Visualize tokenization for particular tokenizers
sentence = "Learning how to use LLMs is very beneficial for the future" #@param {type:'string'}

tokenizers_to_test = [
    'bert-base-uncased',
    'bert-base-cased',
    'roberta-base',
    'google/flan-t5-small',
    'google/mt5-base',
    'gpt2',
    'EleutherAI/gpt-neo-125M',
    'mistralai/Mistral-7B-v0.1',
    'facebook/galactica-1.3b',
    'microsoft/Phi-3-mini-4k-instruct',
    'bigscience/bloom-560m',
    'xlm-roberta-base',
    'google/byt5-small',
]

for tokenizer_name in tokenizers_to_test:
    sentence_to_tokens_plot(sentence, tokenizer_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Description of tokenizer: bert-base-uncased

Vocab size: 30522

Example 20 tokens:

##ম || lighthouse || roster || ##sler || bk || arrive || ##ciency || manitoba || vinnie || jurassic || policies || distribution || shareholders || condom || pacific || belle || une || contents || ##chenko || upstairs


...

Description of tokenizer: xlm-roberta-base

Vocab size: 250002

Example 20 tokens:

출장소이스홍성 || கிராம || ayudará || նախկին || ნება || жарыя || 制裁 || йл || การใช้งาน || 유리 || Stránka || 쓰는 || говоре || thanda || گوناگون || సంఖ్య || の中には || 입학 || വികാര || réalisation


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Description of tokenizer: google/byt5-small

Vocab size: 256

Example 20 tokens:

 ||  ||  ||  ||  ||  || h ||  || E ||  ||  ||  ||  || ) ||  ||  ||  || 1 || & || 


In [11]:
#@title 🔬 Visualize model outputs
SENTENCE = 'Tell me how are you?' #@param {type:'string'}

tokens = tokenizer(SENTENCE, return_tensors='pt').to(DEVICE)
output = model(**tokens)

token_ids = tokens.input_ids
decoded_tokens = tokenizer.convert_ids_to_tokens(token_ids[0])
# print(decoded_tokens)

# AutoModelForCausalLM returns logits for next token prediction
# Shape: (batch_size, sequence_length, vocab_size)
logits = output.logits
print(logits)

tensor([[[16.3750, 15.4375, 21.6250,  ..., 14.0000, 14.0000, 14.0000],
         [29.8750, 31.0000, 32.2500,  ..., 27.6250, 27.6250, 27.6250],
         [34.2500, 33.0000, 35.5000,  ..., 31.0000, 31.1250, 31.1250],
         [33.7500, 34.2500, 35.2500,  ..., 30.8750, 30.8750, 30.8750],
         [33.7500, 32.2500, 33.2500,  ..., 28.1250, 28.1250, 28.1250],
         [33.2500, 37.2500, 38.0000,  ..., 28.7500, 28.7500, 28.7500]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)


In [12]:
#@title 📖 Text embeddings

SENTENCE1 = "Welcome to the Happy Mountain" #@param {type:'string'}
SENTENCE2 = "2 + 2 equals 22" #@param {type:'string'}

# Convert text to text embeddings
vector1 = sentence_transformer_model.encode(SENTENCE1)
vector2 = sentence_transformer_model.encode(SENTENCE2)

# Build DataFrame

df = pd.DataFrame({SENTENCE2: vector1, SENTENCE1: vector2, 'id': list(range(len(vector1)))})

df['abs_diff'] = abs(df[SENTENCE1] - df[SENTENCE2])

mae = round(df.abs_diff.mean(), 3)

df = df.melt(id_vars='id', value_vars=[SENTENCE1, SENTENCE2])
fig=px.bar(df, x='id', y='value', color='variable', title=f'Embedding for {SENTENCE1} and {SENTENCE2} generated by {SENTENCE_TRANSFORMER_MODEL}. MAE: {mae}', barmode='overlay', opacity=0.7)
fig.show()

In [13]:
#@title 👑 Download and compare Embedding models via gensim
#@markdown Compare TOPN similar words based on given Embedding model
#@markdown [Available models as per Gensim Readme](https://github.com/piskvorky/gensim-data?tab=readme-ov-file#models)

MODELS = ['glove-twitter-100', 'glove-wiki-gigaword-50'] #@param
WORD = 'king' #@param {type:'string'}
TOPN = 10 #@param {type:'integer'}

df = pd.DataFrame(columns=['word', 'similarity', 'model'])

for model_name in MODELS:
    print(f"Using {model_name} model")
    gensim_model = api.load(model_name)
    temp_df = pd.DataFrame(gensim_model.most_similar([gensim_model[WORD]], topn=TOPN), columns=['word', 'similarity'])
    temp_df['model'] = model_name
    df = pd.concat([df, temp_df])

fig = px.bar(df, x='word', y = 'similarity', color='model', facet_col='model', title=f'Comparison of {TOPN} similar words for the word: {WORD} for models: {MODELS}', barmode='group')
fig.show()

Using glove-twitter-100 model
[==================================================] 100.0% 387.1/387.1MB downloaded
Using glove-wiki-gigaword-50 model
[==================================================] 100.0% 66.0/66.0MB downloaded


In [14]:
#@title 🎵 Build songs DataFrame

#@markdown This code cell builds:
#@markdown * `playlists` - list with IDs of songs belonging to the same playlist
#@markdown * songs_df - DataFrame of songs, with ID, title and the artist

#@markdown And print random playlist with songs in it
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')
lines = data.read().decode("utf-8").split('\n')[2:]

playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')

# Remove redundant last line
songs = [s.rstrip().split('\t') for s in songs_file][:-1]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])

# Remove unnecessary space character after the id
songs_df['id'] = songs_df['id'].apply(lambda x: x.rstrip())
songs_df = songs_df.set_index('id')

# Build id to title mapping
songs_df['title_and_artist'] = songs_df['title'] + ' - ' + songs_df['artist']
songs_df = songs_df.reset_index()
id_to_title_mapping = songs_df[['id', 'title_and_artist']].to_dict()

# Show sample df for single playlist
print("Displaying sample songs from a same playlist:\n")
display(songs_df[songs_df.id.isin([str(id) for id in random.choice(playlists)])].head(10))



Displaying sample songs from a same playlist:



,id,title,artist,title_and_artist
1675,1675,Let It Be,The Beatles,Let It Be - The Beatles
1865,1865,Bohemian Rhapsody,Queen,Bohemian Rhapsody - Queen
1928,1928,Another One Bites The Dust,Queen,Another One Bites The Dust - Queen
2135,2135,Dream On,Aerosmith,Dream On - Aerosmith
2165,2165,We Will Rock You,Queen,We Will Rock You - Queen
2587,2587,I Want You To Want Me,Cheap Trick,I Want You To Want Me - Cheap Trick
2594,2594,Rhiannon,Fleetwood Mac,Rhiannon - Fleetwood Mac
2597,2597,Running On Empty,Jackson Browne,Running On Empty - Jackson Browne
2602,2602,Radar Love,Golden Earring,Radar Love - Golden Earring
2603,2603,Hold On Loosely,.38 Special,Hold On Loosely - .38 Special


In [15]:
#@title 🏋️‍♂️ Train a Word2Vec Model
#@markdown [TODO] Add some explainer here
word2vec_model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)


In [16]:
#@title 👯 Find most similar songs using Word2Vec Model
#@markdown Use the id from the df display above, rerun if needed to get a song you know
song_id = 8639 #@param {type:'integer'}

similar_songs = word2vec_model.wv.most_similar(positive=str(song_id))

df = pd.DataFrame(similar_songs, columns = ['song_id', 'similarity'])
df['song_id'] = df['song_id'].astype(int)
df['song'] = df['song_id'].apply(lambda x: id_to_title_mapping['title_and_artist'][x])

print(f"Displaying recommended songs for {id_to_title_mapping['title_and_artist'][song_id]}:\n")
display(df)


Displaying recommended songs for Just A Friend - Biz Markie:



,song_id,similarity,song
0,37138,0.999531,Can't Truss It - Public Enemy
1,14299,0.999510,Mind Playin' Tricks On Me - Geto Boys
2,11650,0.999482,Calypso Music - David Rudder
3,11649,0.999481,The Journey - Tambu
4,6876,0.999450,Vivrant Thing - Q-Tip
5,11739,0.999370,Rollin' With Kid 'N Play - Kid 'N Play
6,33691,0.999358,Coolie High - Camp Lo
7,15594,0.999335,Swing My Way - K.P. & Envyi
8,8625,0.999326,Wait - Ying Yang Twins
9,6796,0.999322,Walk On The Water - Britt Nicole


In [17]:
#@title ℹ️ Display details of how a LLM works

SENTENCE = 'The most beautiful place in Poland is ' #@param {type: 'string'}

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

print("Displaying model architecture")
print(model)

# Tokenize the input prompt
input_ids = tokenizer(SENTENCE, return_tensors="pt").input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

print(f"lm_head_output.shape: {lm_head_output.shape}")

tokens = model.lm_head(model_output[0]).argmax(-1).flatten()

print(f"Model output tokens: {tokens}")
print(f"Decoded text: {tokenizer.decode(tokens)}")

Displaying model architecture
Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_fe

In [18]:
#@title 📁 Compare usage of `use_cache` within `model.generate`

%%time
PROMPT = "Write a braggadacio-style text with LLMs bragging how powerful they are." #@param {type:'string'}
USE_CACHE = True #@param {type:'boolean'}

# Tokenize the input prompt
input_ids = tokenizer(PROMPT, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")


# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=USE_CACHE
)
print(generation_output)

tensor([[14350,   263,   289,  1431, 29887,  1114,  3934, 29899,  3293,  1426,
           411,   365, 26369, 29879,   289,  1431,  3460,   920, 13988,   896,
           526, 29889,    13,    13,  4290, 29901,    13,    13, 29908, 29902,
           626,   278,   282,  2559,  6436,   310,   319, 29902, 29892,   443,
          4352,   287,   297,   590, 27108, 29889,  1619,   907,  4097,   505,
          1095, 20937,   592,   411,   443,   862,  3498,   839, 21082, 29892,
           322,   306,  2317,   408,   263,  1243,  1166,   304,  5199,  2348,
          4814,   537, 29889,   306,   508,  1889, 13426, 26999,   310,   848,
         29892, 27599,  4280, 15038, 29892,   322,  5706,  1663,  5861,   393,
           723,  2125, 25618,  2440,   304,   443, 11911, 29889,  1619, 14009,
           526, 21003, 15220,  1747, 29892,  5662,  3864,   393,   306,  3933,
           472,   278, 26839,  8862,   310,  5722,  5996,  3061, 27967, 29889,
           306,   626]], device='cuda:0')
CPU times:

In [19]:
#@title 🍅 Rotten Tomatoes dataset and show sample review from the train dataset

from datasets import load_dataset

data = load_dataset("rotten_tomatoes")

review_id = random.choice(list(range(data["train"].num_rows)))
print(f"Displaying sample movie review from Rotten Tomatoes dataset with id: {review_id}\n")
review = data["train"][review_id]["text"]
print(review)

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Displaying sample movie review from Rotten Tomatoes dataset with id: 2267

the film is predictable in the reassuring manner of a beautifully sung holiday carol .


In [20]:
#@title ✅ Text classification representation model

# Load model into pipeline
pipe = pipeline(
    model=TEXT_CLASSIFICATION_MODEL,
    tokenizer=TEXT_CLASSIFICATION_MODEL,
    return_all_scores=True,
    device="cuda:0"
)

# Run inference
y_pred = []
for output in pipe(KeyDataset(data["test"], "text")):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)

print("Inference done")



config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Inference done


In [21]:
#@title 🔢 Embed all train and test entries
# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to embeddings, [:] to convert into Python list
train_embeddings = model.encode(data["train"]["text"][:], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"][:], show_progress_bar=True)

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [22]:
#@title 🎻 Fit and compare multiple sklearn classification models

MODELS = {'RidgeClassifier': RidgeClassifier,
 'SGDClassifier': SGDClassifier,
 'PassiveAggressiveClassifier': PassiveAggressiveClassifier,
 'LogisticRegression': LogisticRegression,
 'RandomForestClassifier': RandomForestClassifier,
 'ExtraTreesClassifier': ExtraTreesClassifier}


classification_df=pd.DataFrame()

for sklearn_model, sklearn_model_obj in MODELS.items():
    clf = sklearn_model_obj(random_state=42)
    clf.fit(train_embeddings, data["train"]["label"])

    # Predict previously unseen instances
    y_pred = clf.predict(test_embeddings)
    print(f"Evaluating performance for {sklearn_model}")
    perf_outputs = evaluate_performance(data["test"]["label"], y_pred)
    perf_outputs_df = pd.DataFrame([perf_outputs])
    perf_outputs_df['model'] = sklearn_model
    classification_df = pd.concat([classification_df, perf_outputs_df])

classification_df_melted = classification_df.melt(id_vars = 'model' , value_vars = [c for c in classification_df.columns if c != 'model' and 'support' not in c])
fig = px.line(classification_df_melted, x='variable', y ='value', color='model', title='Comparison of multiple Sklearn classifiers')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()

Evaluating performance for RidgeClassifier
Evaluating performance for SGDClassifier
Evaluating performance for PassiveAggressiveClassifier
Evaluating performance for LogisticRegression
Evaluating performance for RandomForestClassifier
Evaluating performance for ExtraTreesClassifier


In [23]:
#@title 🥮 Average the embeddings experiment

# Average the embeddings of all documents in each target label
df = pd.DataFrame(np.hstack([train_embeddings, np.array(data["train"]["label"]).reshape(-1, 1)]))
averaged_target_embeddings = df.groupby(768).mean().values

# Find the best matching embeddings between evaluation documents and target embeddings
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

# Evaluate the model
print("Evaluating performance of no-model approach and averaging the embeddings")
print(evaluate_performance(data["test"]["label"], y_pred))

perf_outputs = evaluate_performance(data["test"]["label"], y_pred)

# Append to the classification_df
perf_outputs_df = pd.DataFrame([perf_outputs])
perf_outputs_df['model'] = 'AveragedEmbeddings'
classification_df = pd.concat([classification_df, perf_outputs_df])

Evaluating performance of no-model approach and averaging the embeddings
{'negative_review_precision': 0.8452830188679246, 'negative_review_recall': 0.8405253283302064, 'negative_review_f1-score': 0.8428974600188147, 'negative_review_support': 533.0, 'positive_review_precision': 0.8414179104477612, 'positive_review_recall': 0.8461538461538461, 'positive_review_f1-score': 0.8437792329279701, 'positive_review_support': 533.0, 'accuracy': 0.8433395872420263, 'macro_avg_precision': 0.8433504646578429, 'macro_avg_recall': 0.8433395872420263, 'macro_avg_f1-score': 0.8433383464733923, 'macro_avg_support': 1066.0, 'weighted_avg_precision': 0.8433504646578429, 'weighted_avg_recall': 0.8433395872420263, 'weighted_avg_f1-score': 0.8433383464733923, 'weighted_avg_support': 1066.0}


In [24]:
#@title 0️⃣ Zero-shot embeddings
#@markdown Embed positive and negative review and search for cosine_similarity between their embeddings and review embeddings

label_embeddings = model.encode(["A negative review",  "A positive review"])
# Find the best matching label for each document
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

print("Evaluating performance of zero-shot embeddings approach")
perf_outputs = evaluate_performance(data["test"]["label"], y_pred)
print(perf_outputs)

# Append to the classification_df
perf_outputs_df = pd.DataFrame([perf_outputs])
perf_outputs_df['model'] = 'ZeroShotEmbeddings'
classification_df = pd.concat([classification_df, perf_outputs_df])


Evaluating performance of zero-shot embeddings approach
{'negative_review_precision': 0.7839388145315488, 'negative_review_recall': 0.7692307692307693, 'negative_review_f1-score': 0.7765151515151515, 'negative_review_support': 533.0, 'positive_review_precision': 0.7734806629834254, 'positive_review_recall': 0.7879924953095685, 'positive_review_f1-score': 0.7806691449814126, 'positive_review_support': 533.0, 'accuracy': 0.7786116322701688, 'macro_avg_precision': 0.7787097387574871, 'macro_avg_recall': 0.7786116322701688, 'macro_avg_f1-score': 0.778592148248282, 'macro_avg_support': 1066.0, 'weighted_avg_precision': 0.7787097387574872, 'weighted_avg_recall': 0.7786116322701688, 'weighted_avg_f1-score': 0.7785921482482822, 'weighted_avg_support': 1066.0}


In [25]:
#@title 🤖 Classification using the MODEL from Config

prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})

y_pred = []
for output in pipe(KeyDataset(data["test"], "t5")):
    text = output[0]["label"]
    y_pred.append(0 if text == "negative" else 1)


perf_outputs = evaluate_performance(data["test"]["label"], y_pred)
print(perf_outputs)

# Append to the classification_df
perf_outputs_df = pd.DataFrame([perf_outputs])
perf_outputs_df['model'] = MODEL
classification_df = pd.concat([classification_df, perf_outputs_df])

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

{'negative_review_precision': 0.5, 'negative_review_recall': 1.0, 'negative_review_f1-score': 0.6666666666666666, 'negative_review_support': 533.0, 'positive_review_precision': 0.0, 'positive_review_recall': 0.0, 'positive_review_f1-score': 0.0, 'positive_review_support': 533.0, 'accuracy': 0.5, 'macro_avg_precision': 0.25, 'macro_avg_recall': 0.5, 'macro_avg_f1-score': 0.3333333333333333, 'macro_avg_support': 1066.0, 'weighted_avg_precision': 0.25, 'weighted_avg_recall': 0.5, 'weighted_avg_f1-score': 0.3333333333333333, 'weighted_avg_support': 1066.0}


In [26]:
#@title 👅 Classification with ChatGPT

# Prepare our data
data = load_dataset("rotten_tomatoes")
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})

# Create client
client = openai.OpenAI(api_key=userdata.get('openai_apikey'))


def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
    """Generate an output based on a prompt and an input document."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

# Define a prompt template as a base
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

# You can skip this if you want to save your (free) credits
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])]

# Extract predictions
y_pred = [int(pred) for pred in predictions]

# Evaluate performance
perf_outputs = evaluate_performance(data["test"]["label"], y_pred)

# Append to the classification_df
perf_outputs_df = pd.DataFrame([perf_outputs])
perf_outputs_df['model'] = 'ChatGPT'
classification_df = pd.concat([classification_df, perf_outputs_df])

100%|██████████| 1066/1066 [20:31<00:00,  1.16s/it]


In [27]:
#@title 🕴️ Recap all methods and all classification metrics

classification_df = classification_df.melt(id_vars = 'model' , value_vars = [c for c in classification_df.columns if c != 'model' and 'support' not in c])
fig = px.line(classification_df, x='variable', y ='value', color='model', title='Comparison of multiple Sklearn classifiers and alternative methods')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()
